In [ ]:
"""CLDNNLike model for RadioML.

# Reference:

- [CONVOLUTIONAL,LONG SHORT-TERM MEMORY, FULLY CONNECTED DEEP NEURAL NETWORKS ]

Adapted from code contributed by Mika.
"""
import os

from keras.models import Model
from keras.layers import Input,Dense,Conv1D,MaxPool1D,ReLU,Dropout,Softmax
from keras.layers import LSTM,CuDNNLSTM,Bidirectional,Flatten,LSTM
from keras.utils.vis_utils import plot_model

def LSTMModel(weights=None,
             input_shape=[1024,2],
             classes=26,
             **kwargs):
    if weights is not None and not (os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), '
                         'or the path to the weights file to be loaded.')

    input = Input(input_shape,name='input')
    x = input

    #LSTM Unit
    x = CuDNNLSTM(units=128,return_sequences = True)(x)
    x = CuDNNLSTM(units=128)(x)

    #DNN
    x = Dense(classes,activation='softmax',name='softmax')(x)

    model = Model(inputs = input,outputs = x)

    # Load weights.
    if weights is not None:
        model.load_weights(weights)

    return model

import keras
if __name__ == '__main__':
    model = LSTMModel(None,input_shape=(1024,2),classes=26)

    adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    plot_model(model, to_file='model.png',show_shapes=True) # print model

    print('models layers:', model.layers)
    print('models config:', model.get_config())
    print('models summary:', model.summary())

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import itertools
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, mean_absolute_error, r2_score

# Show loss curves


def plot_confusion_matrix(cm, title='', cmap=plt.get_cmap("Blues"), labels=[], save_filename=None):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm*100, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=10)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=90, size=12)
    plt.yticks(tick_marks, labels, size=12)
    for i in range(len(tick_marks)):
        for j in range(len(tick_marks)):
            if i != j:
                text = plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10)
            elif i == j:
                color = 'darkorange' if int(np.around(cm[i,j]*100)) == 100 else 'darkorange'
                text = plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10, color=color)

    plt.tight_layout()
    plt.ylabel('True label', fontdict={'size':16,})
    plt.xlabel('Predicted label', fontdict={'size':16,})
    if save_filename is not None:
        plt.savefig(save_filename, format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

def calculate_confusion_matrix(Y, Y_hat, classes):
    n_classes = len(classes)
    conf = np.zeros([n_classes, n_classes])
    confnorm = np.zeros([n_classes, n_classes])

    for k in range(0, Y.shape[0]):
        i = list(Y[k,:]).index(1)
        j = int(np.argmax(Y_hat[k,:]))
        conf[i,j] = conf[i,j] + 1

    for i in range(0, n_classes):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    right = np.sum(np.diag(conf))
    wrong = np.sum(conf) - right
    return confnorm, right, wrong

def calculate_acc_at1snr_from_cm(cm):
    return np.round(np.diag(cm) / np.sum(cm, axis=1), 3)

def calculate_metrics(Y, Y_hat):
    Y_true = np.argmax(Y, axis=1)
    Y_pred = np.argmax(Y_hat, axis=1)
    accuracy = accuracy_score(Y_true, Y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(Y_true, Y_pred, average='weighted')
    mse = mean_squared_error(Y_true, Y_pred)
    mae = mean_absolute_error(Y_true, Y_pred)
    r2 = r2_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

def calculate_acc_cm_each_snr(Y, Y_hat, Z, classes=None, save_figure=True, min_snr=0):
    Z_array = Z[:, 0]
    snrs = sorted(list(set(Z_array)))
    acc = np.zeros(len(snrs))
    acc_mod_snr = np.zeros((len(classes), len(snrs)))

    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }

    i = 0
    for snr in snrs:
        Y_snr = Y[np.where(Z_array == snr)]
        Y_hat_snr = Y_hat[np.where(Z_array == snr)]


        accuracy, precision, recall, f1 = calculate_metrics(Y_snr, Y_hat_snr)
        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)




    # Plot Accuracy, Precision, Recall, F1-Score
    for metric, values in metrics.items():
        plt.figure(figsize=(8, 6))
        plt.plot(snrs, values, label=metric)
        for x, y in zip(snrs, values):
            plt.text(x, y, round(y, 3), ha='center', va='bottom', fontsize=8)
        plt.xlabel("Signal to Noise Ratio")
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} vs SNR")
        plt.legend()
        plt.grid()
        plt.savefig(f'figure/{metric}_vs_snr.pdf', format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()




In [ ]:
# Import all the things we need ---
#   by setting env variables before Keras import you can set up which backend and which GPU it uses
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
# os.environ["THEANO_FLAGS"]  = "device=gpu%d"%(0)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import matplotlib
#matplotlib.use('Tkagg')
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
#from matplotlib import pyplot as plt
import pickle, random, sys,h5py
import keras
import keras.backend as K
from keras.callbacks import LearningRateScheduler,TensorBoard
from keras.regularizers import *
from keras.optimizers import Adam
from keras.models import model_from_json
#from keras.utils.vis_utils import plot_model
import tensorflow as tf
# import keras.backend.tensorflow_backend as KTF
# import mltools
# import rmlmodels.CuDNNLSTMModel as culstm

from keras.utils.np_utils import to_categorical
import pandas as pd

def to_amp_phase(X_train, X_val, X_test, nsamples):
    X_train_cmplx = X_train[:, :,0] + 1j * X_train[:, :,1]
    X_val_cmplx = X_val[:, :,0] + 1j * X_val[:, :,1]
    X_test_cmplx = X_test[:, :,0] + 1j * X_test[:, :,1]

    X_train_amp = np.abs(X_train_cmplx)
    X_train_ang = np.arctan2(X_train[:, :,1], X_train[:, :,0]) / np.pi

    X_train_amp = np.reshape(X_train_amp, (-1, 1, nsamples))
    X_train_ang = np.reshape(X_train_ang, (-1, 1, nsamples))

    X_train = np.concatenate((X_train_amp, X_train_ang), axis=1)
    X_train = np.transpose(np.array(X_train), (0, 2, 1))

    X_val_amp = np.abs(X_val_cmplx)
    X_val_ang = np.arctan2(X_val[:, :,1], X_val[:, :,0]) / np.pi

    X_val_amp = np.reshape(X_val_amp, (-1, 1, nsamples))
    X_val_ang = np.reshape(X_val_ang, (-1, 1, nsamples))

    X_val = np.concatenate((X_val_amp, X_val_ang), axis=1)
    X_val = np.transpose(np.array(X_val), (0, 2, 1))

    X_test_amp = np.abs(X_test_cmplx)
    X_test_ang = np.arctan2(X_test[:, :,1], X_test[:, :,0]) / np.pi

    X_test_amp = np.reshape(X_test_amp, (-1, 1, nsamples))
    X_test_ang = np.reshape(X_test_ang, (-1, 1, nsamples))

    X_test = np.concatenate((X_test_amp, X_test_ang), axis=1)
    X_test = np.transpose(np.array(X_test), (0, 2, 1))
    return (X_train, X_val, X_test)

classes = ['BPSK',
               'QPSK',
               '8PSK',
               '16PSK',
               '32PSK',
               '64PSK',
               '4QAM',
               '8QAM',
               '16QAM',
               '32QAM',
               '64QAM',
               '128QAM',
               '256QAM',
               '2FSK',
               '4FSK',
               '8FSK',
               '16FSK',
               '4PAM',
               '8PAM',
               '16PAM',
               'AM-DSB',
               'AM-DSB-SC',
               'AM-USB',
               'AM-LSB',
                'FM',
                'PM']


In [ ]:
##traindata

data1 = h5py.File('Dataset/HisarMod2019.1/Train/train.mat','r')
train=data1['data_save'][:]
train=train.swapaxes(0,2)
train=train.swapaxes(1,2)

data2 = h5py.File('Dataset/HisarMod2019.1/Test/test.mat','r')
test=data2['data_save'][:]
test=test.swapaxes(0,2)
test=test.swapaxes(1,2)


In [ ]:
##label
train_labels = pd.read_csv('Dataset/HisarMod2019.1/Train/train_labels1.csv',header=None)
train_labels=np.array(train_labels)
train_labels = to_categorical(train_labels, num_classes=None)

test_labels = pd.read_csv('Dataset/HisarMod2019.1/Test/test_labels1.csv',header=None)
test_labels =np.array(test_labels)
test_labels = to_categorical(test_labels, num_classes=None)

##snr
train_snr=pd.read_csv('Dataset/HisarMod2019.1/Train/train_snr.csv',header=None)
train_snr=np.array(train_snr)

test_snr=pd.read_csv('Dataset/HisarMod2019.1/Test/test_snr.csv',header=None)
test_snr=np.array(test_snr)

In [ ]:
n_examples = train.shape[0]
n_train = int(n_examples * 0.8)
n_val = int(n_examples * 0.2)
train_idx = list(np.random.choice(range(0, n_examples), size=n_train, replace=False))
val_idx = list(set(range(0, n_examples)) - set(train_idx))
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)
X_train = train[train_idx]
Y_train = train_labels[train_idx]
X_val = train[val_idx]
Y_val = train_labels[val_idx]
X_test = test
Y_test = test_labels
Z_test = test_snr
X_train,X_val,X_test = to_amp_phase(X_train,X_val,X_test,1024)

In [ ]:
# Set up some params
nb_epoch =  200  # number of epochs to train on
batch_size = 300  # training batch size

In [ ]:
model = LSTMModel()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='model_LSTM.png',show_shapes=True) # print model

In [ ]:
filepath = 'weights/weights.h5'
import time
TRS_LSTM=time.time()
history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
    validation_data=(X_val,Y_val),
    callbacks = [
                keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
                keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,verbose=1,patince=5,min_lr=0.000001),
                keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
                #keras.callbacks.TensorBoard(log_dir='./logs/',histogram_freq=1,write_graph=False,write_grads=1,write_images=False,update_freq='epoch')
                ]
                    )

In [ ]:
TRE_LSTM=time.time()
T_LSTM=TRE_LSTM-TRS_LSTM

In [ ]:
TES_LSTM=time.time()

In [ ]:
#Show simple version of performance
score = model.evaluate(X_test, Y_test, verbose=1, batch_size=batch_size)
print(score)

In [ ]:
 calculate_acc_cm_each_snr(Y_test, test_Y_hat, Z_test, classes, min_snr=-18)

In [ ]:
TEE_LSTM=time.time()
T_LSTM_E=TEE_LSTM-TES_LSTM